In [1]:
import sys
import pickle
import numpy as np
import pandas as pd
import networkx as nx
import seaborn as sns
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn.metrics import confusion_matrix

In [2]:
#loading the feature matrix and labels 
feature_matrix = np.load('D:/NLP Project/Data/feature_matrix.npy')
class_labels = np.load('D:/NLP Project/Data/class_lables.npy')
indices = np.arange(len(feature_matrix))

In [3]:
graph_df = pd.read_csv('D:/NLP Project/Data/edge_list.csv')
#loading the edgelist and constructing the url graph
g = nx.from_pandas_edgelist(graph_df,'url1','url2',edge_attr='edge_weight')

In [7]:
nodes = list(g.nodes)
len(nodes)

17601

In [8]:
iter_labels = class_labels #making a copy of class labels for updations during iteration
# Contains true labels of all documents.

In [9]:
np.unique(class_labels,return_counts=True)

(array([0, 1, 2, 3]), array([4440, 4236, 5143, 3782], dtype=int64))

In [10]:
nx.info(g)

'Graph with 17601 nodes and 45259074 edges'

### Model 1 : Naive Bayes

In [11]:
testSize = [0.8,0.6,0.4,0.2]
accuracy_list_NB = []
precision_list_NB = []
recall_list_NB = []
conf_mat_NB = []

#loading the feature matrix and labels 
feature_matrix = np.load('D:/NLP Project/Data/feature_matrix.npy')
class_labels = np.load('D:/NLP Project/Data/class_lables.npy')
indices = np.arange(len(feature_matrix))

for k in testSize:
    print('For test size: ',k)
    X_train, X_test, y_train, y_test = train_test_split(feature_matrix, class_labels,test_size=k, random_state=2, stratify=class_labels)
    clf = GaussianNB() # Bootstrapping using Naive Bayes as Base Classifier
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    confusion_mat = confusion_matrix(y_test,y_pred)

    conf_mat_NB.append(confusion_mat)
    accuracy_list_NB.append(metrics.accuracy_score(y_test,y_pred))
    precision_list_NB.append(metrics.precision_score(y_test,y_pred,average='macro',zero_division=0))
    recall_list_NB.append(metrics.recall_score(y_test,y_pred,average='macro'))

    # Micro
    precision_list_NB.append(metrics.precision_score(y_test,y_pred,average='micro',zero_division=0))
    recall_list_NB.append(metrics.recall_score(y_test,y_pred,average='micro'))


For test size:  0.8
For test size:  0.6
For test size:  0.4
For test size:  0.2


In [12]:
accuracy_list_NB = ["%.2f" % elem for elem in accuracy_list_NB]
precision_list_NB = ["%.2f" % elem for elem in precision_list_NB]
recall_list_NB = ["%.2f" % elem for elem in recall_list_NB]

print(accuracy_list_NB)
print(precision_list_NB)
print(recall_list_NB)
print(conf_mat_NB)

['0.66', '0.64', '0.65', '0.65']
['0.66', '0.66', '0.65', '0.64', '0.68', '0.65', '0.68', '0.65']
['0.66', '0.66', '0.65', '0.64', '0.67', '0.65', '0.67', '0.65']
[array([[2390,  427,  588,  147],
       [ 524, 2020,  457,  388],
       [ 942,  604, 2403,  165],
       [  61,  422,  127, 2416]], dtype=int64), array([[2012,  182,  310,  160],
       [ 536, 1280,  264,  462],
       [1040,  236, 1594,  216],
       [  51,  246,   71, 1901]], dtype=int64), array([[1442,   82,  114,  138],
       [ 353,  795,  135,  412],
       [ 751,  131, 1000,  175],
       [  22,  110,   25, 1356]], dtype=int64), array([[720,  32,  65,  71],
       [173, 402,  61, 211],
       [387,  65, 478,  99],
       [  8,  31,  14, 704]], dtype=int64)]


### Model 1 : ICA - NB with Label Counts

In [13]:
##Bootstrapping
testSize = [0.8,0.6,0.4,0.2]
accuracy_list_ICA_Labels = []
precision_list_ICA_Labels = []
recall_list_ICA_Labels = []
conf_mat_ICA_Labels = []

feature_matrix = np.load('D:/NLP Project/Data/feature_matrix.npy')
class_labels = np.load('D:/NLP Project/Data/class_lables.npy')
indices = np.arange(len(feature_matrix))

for k in testSize:
    print('For test size: ',k)
    X_train, X_test, y_train, y_test,idx_train,idx_test = train_test_split(feature_matrix, class_labels, indices,test_size=k, random_state=2, stratify=class_labels)
    clf = GaussianNB() # Bootstrapping using Naive Bayes as Base Classifier
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test) # y-pred is bootstrapped labels
    iter_labels = class_labels
    np.put(iter_labels,idx_test,y_pred)  # updating labels of test data with the predecited labels 
    iter_labels_list = list(iter_labels)
    distinct_labels = sorted(list(set(iter_labels_list)))

    # ICA starting

    adj_feats_train = np.zeros((len(X_train),len(distinct_labels)))
    adj_feats_test  = np.zeros((len(X_test),len(distinct_labels)))

    ########################################################################
    ##constructing additional features, train and Iterate until stabilized##

    #updating the adjacent features for training nodes
    for i in range(len(idx_train)):
        adjs = list(g[idx_train[i]])
        adjs = [int(nei) for nei in adjs]
        labels_of_adjacent_nodes = [iter_labels_list[nei] for nei in adjs]

        for j in range(len(distinct_labels)):
            this_label = distinct_labels[j]
            cnt_of_adjacent_labels = labels_of_adjacent_nodes.count(this_label) ## Voting by neighbor nodes.
            adj_feats_train[i][this_label] = cnt_of_adjacent_labels
        
          
    X_train_updated = np.concatenate((X_train,adj_feats_train),axis = 1)

    #updating the adjancent features for test nodes
    for i in range(len(idx_test)):
        adjs = list(g[idx_test[i]])
        adjs = [int(nei) for nei in adjs]
        labels_of_adjacent_nodes = [iter_labels_list[nei] for nei in adjs]

        for j in range(len(distinct_labels)):
            this_label = distinct_labels[j]
            cnt_of_adjacent_labels = labels_of_adjacent_nodes.count(this_label) ## Voting by neighbor nodes.
            adj_feats_test[i][this_label] = cnt_of_adjacent_labels

    X_test_updated = np.concatenate((X_test,adj_feats_test),axis = 1)  

    #learning the new model on updated feature matrix with adjacent labels
    clf_updated = GaussianNB()
    clf_updated.fit(X_train_updated,y_train)
    #print("\nStarting ICA Loop: ...\n")
    #staring the ICA inference loop
    loop_var = 0
    iter_var = 0
    y_pred_current = y_pred
    while (loop_var == 0 and iter_var < 15):
            y_pred_updated = clf_updated.predict(X_test_updated)
            if(np.array_equal(y_pred_current, y_pred_updated)):
                #algorithm stabilized
                #print("ICA Stabilized")
                loop_var = 1        
            else:
                loop_var = 0
                iter_var += 1
                #print("ICA Loop: "+str(iter_var))
                
                #updating the labels for test nodes with new predictions
                for i in range(len(idx_test)):
                    iter_labels_list[idx_test[i]] = y_pred_updated[i]
                
                #updating the adjacent features for test nodes
                for i in range(len(idx_test)):
                    adjs = list(g[idx_test[i]])
                    adjs = [int(nei) for nei in adjs]
                    labels_of_adjacent_nodes = [iter_labels_list[nei] for nei in adjs]

                    for j in range(len(distinct_labels)):
                        this_label = distinct_labels[j]
                        cnt_of_adjacent_labels = labels_of_adjacent_nodes.count(this_label) ## Voting by neighbor nodes.
                        adj_feats_test[i][this_label] = cnt_of_adjacent_labels
                                        
                X_test_updated = np.concatenate((X_test,adj_feats_test),axis = 1)
                y_pred_current = y_pred_updated   
    
    #print('No. of iterations ICA ran: ',iter_var)            
    final_predictions = y_pred_updated

    #print('\nMetrics for Iterative Classification Algorithm for train size {:.1f}:\n'.format(1-k))
    # ICA
    # Macro
    confusion_mat = confusion_matrix(y_test,y_pred_updated)
    conf_mat_ICA_Labels.append(confusion_mat)

    accuracy_list_ICA_Labels.append(metrics.accuracy_score(y_test,y_pred_updated))
    precision_list_ICA_Labels.append(metrics.precision_score(y_test,y_pred_updated,average='macro',zero_division=0))
    recall_list_ICA_Labels.append(metrics.recall_score(y_test,y_pred_updated,average='macro'))

    # Micro
    precision_list_ICA_Labels.append(metrics.precision_score(y_test,y_pred_updated,average='micro',zero_division=0))
    recall_list_ICA_Labels.append(metrics.recall_score(y_test,y_pred_updated,average='micro'))


For test size:  0.8
For test size:  0.6
For test size:  0.4
For test size:  0.2


In [14]:
accuracy_list_ICA_Labels = ["%.2f" % elem for elem in accuracy_list_ICA_Labels]
precision_list_ICA_Labels = ["%.2f" % elem for elem in precision_list_ICA_Labels]
recall_list_ICA_Labels = ["%.2f" % elem for elem in recall_list_ICA_Labels]

print(accuracy_list_ICA_Labels)
print(precision_list_ICA_Labels)
print(recall_list_ICA_Labels)
print(conf_mat_ICA_Labels)

['0.74', '0.78', '0.80', '0.81']
['0.74', '0.74', '0.78', '0.78', '0.80', '0.80', '0.81', '0.81']
['0.74', '0.74', '0.78', '0.78', '0.79', '0.80', '0.80', '0.81']
[array([[2665,  286,  547,   54],
       [ 386, 2265,  438,  300],
       [ 697,  475, 2831,  111],
       [  44,  299,   62, 2621]], dtype=int64), array([[2476,  170,  184,   53],
       [ 302, 1824,  201,  265],
       [ 525,  251, 1866,  121],
       [ 122,  128,   25, 2048]], dtype=int64), array([[1907,  107,   98,   39],
       [ 173, 1112,   98,  191],
       [ 333,  121, 1078,   98],
       [  92,   75,   16, 1503]], dtype=int64), array([[1006,   65,   55,   21],
       [  62,  536,   25,  104],
       [ 162,   40,  505,   43],
       [  52,   35,    6,  804]], dtype=int64)]


### Model 2 : ICA - NB with Sum of Edge Weights 

In [15]:
testSize = [0.8,0.6,0.4,0.2]
accuracy_list_ICA_EW= []
precision_list_ICA_EW = []
recall_list_ICA_EW = []
conf_mat_ICA_EW = []

feature_matrix = np.load('D:/NLP Project/Data/feature_matrix.npy')
class_labels = np.load('D:/NLP Project/Data/class_lables.npy')
indices = np.arange(len(feature_matrix))

for k in testSize:
    print('For test size: ',k)
    X_train, X_test, y_train, y_test,idx_train,idx_test = train_test_split(feature_matrix, class_labels, indices,test_size=k, random_state=2, stratify=class_labels)
    clf = GaussianNB()
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    iter_labels = class_labels
    np.put(iter_labels,idx_test,y_pred)  # updating labels of test data with the predecited labels 
    iter_labels_list = list(iter_labels)
    distinct_labels = sorted(list(set(iter_labels_list)))

    # ICA starting

    adj_feats_train = np.zeros((len(X_train),len(distinct_labels)))
    adj_feats_test  = np.zeros((len(X_test),len(distinct_labels)))

    ########################################################################
    ##constructing additional features, train and Iterate until stabilized##

    #updating the adjancent features for training nodes
    for i in range(len(idx_train)):
        adjs = list(g[idx_train[i]])
        adjs = [int(nei) for nei in adjs]
        edge_weight_adj_nodes = [0]*len(distinct_labels)

        for j in range(len(adjs)):
            this_label = iter_labels_list[adjs[j]]
            edge_weight_adj_nodes[this_label] +=  int(g[idx_train[i]][adjs[j]]['edge_weight'])/len(g[idx_train[i]])
            adj_feats_train[i][this_label] = edge_weight_adj_nodes[this_label]
        
          
    X_train_updated = np.concatenate((X_train,adj_feats_train),axis = 1)

    #updating the adjancent features for test nodes
    for i in range(len(idx_test)):
        adjs = list(g[idx_test[i]])
        adjs = [int(nei) for nei in adjs]
        edge_weight_adj_nodes = [0]*len(distinct_labels)

        for j in range(len(adjs)):
            this_label = iter_labels_list[adjs[j]]
            edge_weight_adj_nodes[this_label] +=  int(g[idx_test[i]][adjs[j]]['edge_weight'])/len(g[idx_test[i]])
            adj_feats_test[i][this_label] = edge_weight_adj_nodes[this_label]

    X_test_updated = np.concatenate((X_test,adj_feats_test),axis = 1)  

    #learning the new model on updated feature matrix with adjacent labels
    clf_updated = GaussianNB()
    clf_updated.fit(X_train_updated,y_train)
    #print("\nStarting ICA Loop: ...\n")
    #staring the ICA inference loop
    loop_var = 0
    iter_var = 0
    y_pred_current = y_pred
    while (loop_var == 0 and iter_var < 15):
            y_pred_updated = clf_updated.predict(X_test_updated)
            if(np.array_equal(y_pred_current, y_pred_updated)):
                #algorithm stabilized
                #print("ICA Stabilized")
                loop_var = 1        
            else:
                loop_var = 0
                iter_var += 1
                #print("ICA Loop: "+str(iter_var))
                
                #updating the labels for test nodes with new predictions
                for i in range(len(idx_test)):
                    iter_labels_list[idx_test[i]] = y_pred_updated[i]
                
                #updating the adjancent features for test nodes
                for i in range(len(idx_test)):
                    adjs = list(g[idx_test[i]])
                    adjs = [int(nei) for nei in adjs]
                    edge_weight_adj_nodes = [0]*len(distinct_labels)
    
                    for j in range(len(adjs)):
                        this_label = iter_labels_list[adjs[j]]
                        edge_weight_adj_nodes[this_label] +=  int(g[idx_test[i]][adjs[j]]['edge_weight'])/len(g[idx_test[i]])
                        adj_feats_test[i][this_label] = edge_weight_adj_nodes[this_label]     
                    
                X_test_updated = np.concatenate((X_test,adj_feats_test),axis = 1)
                y_pred_current = y_pred_updated   
    #print('No. of iterations ICA ran: ',iter_var)            
    final_predictions = y_pred_updated

    #print('ICA - Edge Weight Confusion matrix:\n')
    confusion_mat = confusion_matrix(y_test,y_pred_updated)
    conf_mat_ICA_EW.append(confusion_mat)
    #print(confusion_mat)

    #print('\nMetrics for Iterative Classification Algorithm for train size {:.1f}:\n'.format(1-k))
    # ICA
    # Macro
    accuracy_list_ICA_EW.append(metrics.accuracy_score(y_test,y_pred_updated))
    precision_list_ICA_EW.append(metrics.precision_score(y_test,y_pred_updated,average='macro',zero_division=0))
    recall_list_ICA_EW.append(metrics.recall_score(y_test,y_pred_updated,average='macro'))

    # Micro
    precision_list_ICA_EW.append(metrics.precision_score(y_test,y_pred_updated,average='micro',zero_division=0))
    recall_list_ICA_EW.append(metrics.recall_score(y_test,y_pred_updated,average='micro'))


For test size:  0.8
For test size:  0.6
For test size:  0.4
For test size:  0.2


In [16]:
accuracy_list_ICA_EW = ["%.2f" % elem for elem in accuracy_list_ICA_EW]
precision_list_ICA_EW = ["%.2f" % elem for elem in precision_list_ICA_EW]
recall_list_ICA_EW = ["%.2f" % elem for elem in recall_list_ICA_EW]

print(accuracy_list_ICA_EW)
print(precision_list_ICA_EW)
print(recall_list_ICA_EW)
print(conf_mat_ICA_EW)

['0.66', '0.74', '0.82', '0.86']
['0.66', '0.66', '0.74', '0.74', '0.83', '0.82', '0.87', '0.86']
['0.66', '0.66', '0.74', '0.74', '0.81', '0.82', '0.84', '0.86']
[array([[2390,  427,  588,  147],
       [ 524, 2020,  457,  388],
       [ 942,  603, 2404,  165],
       [  61,  422,  127, 2416]], dtype=int64), array([[2407,  153,  196,  127],
       [ 390, 1597,  218,  387],
       [ 602,  278, 1748,  135],
       [  56,  178,   72, 2017]], dtype=int64), array([[1980,   39,   56,   76],
       [ 190, 1069,  105,  210],
       [ 313,  113, 1126,   78],
       [  27,   53,   16, 1590]], dtype=int64), array([[1093,   13,   13,   28],
       [  64,  548,   17,   98],
       [ 123,   59,  525,   43],
       [  18,    6,    2,  871]], dtype=int64)]


### Model 3 : ICA-NB with Combined Features

In [17]:
testSize = [0.8,0.6,0.4,0.2]
accuracy_list_ICA_Combined = []
precision_list_ICA_Combined = []
recall_list_ICA_Combined = []
conf_matrix_ICA_Combined = []

feature_matrix = np.load('D:/NLP Project/Data/feature_matrix.npy')
class_labels = np.load('D:/NLP Project/Data/class_lables.npy')
indices = np.arange(len(feature_matrix))

for k in testSize:
    print('For test size: ',k)
    X_train, X_test, y_train, y_test,idx_train,idx_test = train_test_split(feature_matrix, class_labels, indices,test_size=k, random_state=2, stratify=class_labels)
    clf = GaussianNB()
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    iter_labels = class_labels
    np.put(iter_labels,idx_test,y_pred)  # updating labels of test data with the predecited labels 
    iter_labels_list = list(iter_labels)
    distinct_labels = sorted(list(set(iter_labels_list)))

    # ICA starting

    adj_feats_train = np.zeros((len(X_train),2*len(distinct_labels))) # Multiply by 2, since we have 8 additional features combined.
    adj_feats_test  = np.zeros((len(X_test),2*len(distinct_labels)))

    ########################################################################
    ##constructing additional features, train and Iterate until stabilized##

    #updating the adjancent features for training nodes
    for i in range(len(idx_train)):       
        adjs = list(g[idx_train[i]])
        adjs = [int(nei) for nei in adjs]
        edge_weight_adj_nodes = [0]*len(distinct_labels)
        labels_of_adjacent_nodes = [iter_labels_list[nei] for nei in adjs]
        
        for j in range(len(adjs)):
            this_label = iter_labels_list[adjs[j]]
            edge_weight_adj_nodes[this_label] +=  int(g[idx_train[i]][adjs[j]]['edge_weight'])/len(g[idx_train[i]])
            adj_feats_train[i][this_label] = edge_weight_adj_nodes[this_label]
        
        for k in range(len(distinct_labels)):
            this_label = distinct_labels[k]
            cnt_of_adjacent_labels = labels_of_adjacent_nodes.count(this_label) ## Voting by neighbor nodes.
            adj_feats_train[i][this_label+len(distinct_labels)] = cnt_of_adjacent_labels
          
    X_train_updated = np.concatenate((X_train,adj_feats_train),axis = 1)

    #updating the adjancent features for test nodes
    for i in range(len(idx_test)):       
        adjs = list(g[idx_test[i]])
        adjs = [int(nei) for nei in adjs]
        edge_weight_adj_nodes = [0]*len(distinct_labels)
        labels_of_adjacent_nodes = [iter_labels_list[nei] for nei in adjs]
        
        for j in range(len(adjs)):
            this_label = iter_labels_list[adjs[j]]
            edge_weight_adj_nodes[this_label] +=  int(g[idx_test[i]][adjs[j]]['edge_weight'])/len(g[idx_test[i]])
            adj_feats_test[i][this_label] = edge_weight_adj_nodes[this_label]
        
        for k in range(len(distinct_labels)):
            this_label = distinct_labels[k]
            cnt_of_adjacent_labels = labels_of_adjacent_nodes.count(this_label) ## Voting by neighbor nodes.
            adj_feats_test[i][this_label+len(distinct_labels)] = cnt_of_adjacent_labels

    X_test_updated = np.concatenate((X_test,adj_feats_test),axis = 1)  

    #learning the new model on updated feature matrix with adjacent labels
    clf_updated = GaussianNB()
    clf_updated.fit(X_train_updated,y_train)
    #print("\nStarting ICA Loop: ...\n")
    #staring the ICA inference loop
    loop_var = 0
    iter_var = 0
    y_pred_current = y_pred
    while (loop_var == 0 and iter_var < 15):
            y_pred_updated = clf_updated.predict(X_test_updated)
            if(np.array_equal(y_pred_current, y_pred_updated)):
                #algorithm stabilized
                #print("ICA Stabilized")
                loop_var = 1        
            else:
                loop_var = 0
                iter_var += 1
                #print("ICA Loop: "+str(iter_var))
                
                #updating the labels for test nodes with new predictions
                for i in range(len(idx_test)):
                    iter_labels_list[idx_test[i]] = y_pred_updated[i]
                
                #updating the adjancent features for test nodes
                for i in range(len(idx_test)):       
                    adjs = list(g[idx_test[i]])
                    adjs = [int(nei) for nei in adjs]
                    edge_weight_adj_nodes = [0]*len(distinct_labels)
                    labels_of_adjacent_nodes = [iter_labels_list[nei] for nei in adjs]
                    
                    for j in range(len(adjs)):
                        this_label = iter_labels_list[adjs[j]]
                        edge_weight_adj_nodes[this_label] +=  int(g[idx_test[i]][adjs[j]]['edge_weight'])/len(g[idx_test[i]])
                        adj_feats_test[i][this_label] = edge_weight_adj_nodes[this_label]
                    
                    for k in range(len(distinct_labels)):
                        this_label = distinct_labels[k]
                        cnt_of_adjacent_labels = labels_of_adjacent_nodes.count(this_label) ## Voting by neighbor nodes.
                        adj_feats_test[i][this_label+len(distinct_labels)] = cnt_of_adjacent_labels       
                                
                X_test_updated = np.concatenate((X_test,adj_feats_test),axis = 1)
                y_pred_current = y_pred_updated   
    print('No. of iterations ICA ran: ',iter_var)            
    final_predictions = y_pred_updated

    #print('ICA - Edge Weight Confusion matrix:\n')
    confusion_mat = confusion_matrix(y_test,y_pred_updated)
    conf_matrix_ICA_Combined.append(confusion_mat)
    #print(confusion_mat)

    #print('\nMetrics for Iterative Classification Algorithm for train size {:.1f}:\n'.format(1-k))
    # ICA
    # Macro
    accuracy_list_ICA_Combined.append(metrics.accuracy_score(y_test,y_pred_updated))
    precision_list_ICA_Combined.append(metrics.precision_score(y_test,y_pred_updated,average='macro',zero_division=0))
    recall_list_ICA_Combined.append(metrics.recall_score(y_test,y_pred_updated,average='macro'))

    # Micro
    precision_list_ICA_Combined.append(metrics.precision_score(y_test,y_pred_updated,average='micro',zero_division=0))
    recall_list_ICA_Combined.append(metrics.recall_score(y_test,y_pred_updated,average='micro'))


For test size:  0.8
No. of iterations ICA ran:  2
For test size:  0.6
No. of iterations ICA ran:  2
For test size:  0.4
No. of iterations ICA ran:  1
For test size:  0.2
No. of iterations ICA ran:  1


In [18]:
accuracy_list_ICA_Combined = ["%.2f" % elem for elem in accuracy_list_ICA_Combined]
precision_list_ICA_Combined = ["%.2f" % elem for elem in precision_list_ICA_Combined]
recall_list_ICA_Combined = ["%.2f" % elem for elem in recall_list_ICA_Combined]

print(accuracy_list_ICA_Combined)
print(precision_list_ICA_Combined)
print(recall_list_ICA_Combined)
print(conf_matrix_ICA_Combined)

['0.74', '0.78', '0.80', '0.81']
['0.74', '0.74', '0.78', '0.78', '0.80', '0.80', '0.81', '0.81']
['0.74', '0.74', '0.78', '0.78', '0.79', '0.80', '0.80', '0.81']
[array([[2666,  286,  546,   54],
       [ 385, 2266,  438,  300],
       [ 697,  475, 2831,  111],
       [  43,  300,   62, 2621]], dtype=int64), array([[2474,  171,  185,   53],
       [ 303, 1822,  201,  266],
       [ 524,  251, 1867,  121],
       [ 122,  128,   25, 2048]], dtype=int64), array([[1907,  107,   98,   39],
       [ 173, 1113,   98,  190],
       [ 333,  122, 1077,   98],
       [  93,   75,   16, 1502]], dtype=int64), array([[1004,   67,   55,   21],
       [  62,  537,   24,  104],
       [ 163,   41,  504,   42],
       [  53,   35,    6,  803]], dtype=int64)]
